# Challenge 3

In this challenge we will work on the `Orders` data set. In your work you will apply the thinking process and workflow we showed you in Challenge 2.

You are serving as a Business Intelligence Analyst at the headquarter of an international fashion goods chain store. Your boss today asked you to do two things for her:

**First, identify two groups of customers from the data set.** The first group is **VIP Customers** whose **aggregated expenses** at your global chain stores are **above the 95th percentile** (aka. 0.95 quantile). The second group is **Preferred Customers** whose **aggregated expenses** are **between the 75th and 95th percentile**.

**Second, identify which country has the most of your VIP customers, and which country has the most of your VIP+Preferred Customers combined.**

## Q1: How to identify VIP & Preferred Customers?

We start by importing all the required libraries:

In [7]:
# import required libraries
import numpy as np
import pandas as pd

Next, extract and import `Orders` dataset into a dataframe variable called `orders`. Print the head of `orders` to overview the data:

In [12]:
import zipfile
import pandas as pd

zip_path = r'C:\Users\Tayna Appel\Documents\GitHub\Aulas - Ironhack\Labs\lab-pandas-calculations\your-code\Orders.zip'
extract_path = r'C:\Users\Tayna Appel\Documents\GitHub\Aulas - Ironhack\Labs\lab-pandas-calculations\your-code'

# Extrair conteúdo do ZIP
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

# Agora abre o CSV
orders = pd.read_csv(extract_path + r'\Orders.csv')
print(orders.head())


   Unnamed: 0  InvoiceNo StockCode  year  month  day  hour  \
0           0     536365    85123A  2010     12    3     8   
1           1     536365     71053  2010     12    3     8   
2           2     536365    84406B  2010     12    3     8   
3           3     536365    84029G  2010     12    3     8   
4           4     536365    84029E  2010     12    3     8   

                           Description  Quantity          InvoiceDate  \
0   white hanging heart t-light holder         6  2010-12-01 08:26:00   
1                  white metal lantern         6  2010-12-01 08:26:00   
2       cream cupid hearts coat hanger         8  2010-12-01 08:26:00   
3  knitted union flag hot water bottle         6  2010-12-01 08:26:00   
4       red woolly hottie white heart.         6  2010-12-01 08:26:00   

   UnitPrice  CustomerID         Country  amount_spent  
0       2.55       17850  United Kingdom         15.30  
1       3.39       17850  United Kingdom         20.34  
2       2.75     

---

"Identify VIP and Preferred Customers" is the non-technical goal of your boss. You need to translate that goal into technical languages that data analysts use:

## How to label customers whose aggregated `amount_spent` is in a given quantile range?


We break down the main problem into several sub problems:

#### Sub Problem 1: How to aggregate the  `amount_spent` for unique customers?

#### Sub Problem 2: How to select customers whose aggregated `amount_spent` is in a given quantile range?

#### Sub Problem 3: How to label selected customers as "VIP" or "Preferred"?

*Note: If you want to break down the main problem in a different way, please feel free to revise the sub problems above.*

Now in the workspace below, tackle each of the sub problems using the iterative problem solving workflow. Insert cells as necessary to write your codes and explain your steps.

In [15]:
customer_spending = orders.groupby('CustomerID')['amount_spent'].sum().reset_index()
customer_spending.rename(columns={'amount_spent': 'total_spent'}, inplace=True)

print(customer_spending.head(20))

    CustomerID  total_spent
0        12346     77183.60
1        12347      4310.00
2        12348      1797.24
3        12349      1757.55
4        12350       334.40
5        12352      2506.04
6        12353        89.00
7        12354      1079.40
8        12355       459.40
9        12356      2811.43
10       12357      6207.67
11       12358      1168.06
12       12359      6372.58
13       12360      2662.06
14       12361       189.90
15       12362      5226.23
16       12363       552.00
17       12364      1313.10
18       12365       641.38
19       12367       168.90


In [16]:
q90 = customer_spending['total_spent'].quantile(0.90)
q70 = customer_spending['total_spent'].quantile(0.70)

def classify_customer(spent):
    if spent >= q90:
        return 'VIP'
    elif spent >= q70:
        return 'Preferred'
    else:
        return 'Regular'

customer_spending['label'] = customer_spending['total_spent'].apply(classify_customer)

print(customer_spending.head())

   CustomerID  total_spent      label
0       12346     77183.60        VIP
1       12347      4310.00        VIP
2       12348      1797.24  Preferred
3       12349      1757.55  Preferred
4       12350       334.40    Regular


In [17]:
print(customer_spending['label'].value_counts())

orders = orders.merge(customer_spending[['CustomerID', 'label']], on='CustomerID', how='left')
orders

label
Regular      3036
Preferred     869
VIP           434
Name: count, dtype: int64


,Unnamed: 0,InvoiceNo,StockCode,year,month,day,hour,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,amount_spent,label
0,0,536365,85123A,2010,12,3,8,white hanging heart t-light holder,6,2010-12-01 08:26:00,2.55,17850,United Kingdom,15.30,VIP
1,1,536365,71053,2010,12,3,8,white metal lantern,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34,VIP
2,2,536365,84406B,2010,12,3,8,cream cupid hearts coat hanger,8,2010-12-01 08:26:00,2.75,17850,United Kingdom,22.00,VIP
3,3,536365,84029G,2010,12,3,8,knitted union flag hot water bottle,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34,VIP
4,4,536365,84029E,2010,12,3,8,red woolly hottie white heart.,6,2010-12-01 08:26:00,3.39,17850,United Kingdom,20.34,VIP
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
397919,541904,581587,22613,2011,12,5,12,pack of 20 spaceboy napkins,12,2011-12-09 12:50:00,0.85,12680,France,10.20,Regular
397920,541905,581587,22899,2011,12,5,12,children's apron dolly girl,6,2011-12-09 12:50:00,2.10,12680,France,12.60,Regular
397921,541906,581587,23254,2011,12,5,12,childrens cutlery dolly girl,4,2011-12-09 12:50:00,4.15,12680,France,16.60,Regular
397922,541907,581587,23255,2011,12,5,12,childrens cutlery circus parade,4,2011-12-09 12:50:00,4.15,12680,France,16.60,Regular


Now we'll leave it to you to solve Q2 & Q3, which you can leverage from your solution for Q1:

## Q2: How to identify which country has the most VIP Customers?

In [20]:
vip_customers = orders[orders['label'] == 'VIP']
vip_by_country = vip_customers['Country'].value_counts()

top_vip_country = vip_by_country.idxmax()
vip_count = vip_by_country.max
vip_count

<bound method Series.max of Country
United Kingdom     131497
EIRE                 7238
Germany              5311
France               5111
Netherlands          2080
Spain                1506
Switzerland          1028
Australia             898
Norway                758
Portugal              681
Belgium               680
Channel Islands       364
Finland               331
Cyprus                248
Singapore             222
Japan                 209
Sweden                198
Iceland               182
Israel                171
Poland                149
Denmark               112
Austria                21
Name: count, dtype: int64>

## Q3: How to identify which country has the most VIP+Preferred Customers combined?

In [24]:
vip_preferred_customers = orders[orders['label'].isin(['VIP', 'Preferred'])]
vip_pref_by_country = vip_preferred_customers['Country'].value_counts()

top_country = vip_pref_by_country.idxmax()
total_top = vip_pref_by_country.max()

top_country, total_top

('United Kingdom', 243879)